# Predict Future Sales
You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.

## File descriptions
* sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
* test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
* sample_submission.csv - a sample submission file in the correct format.
* items.csv - supplemental information about the items/products.
* item_categories.csv  - supplemental information about the items categories.
* shops.csv- supplemental information about the shops.

In [1]:
import pandas as pd

items=r'/kaggle/input/competitive-data-science-predict-future-sales/items.csv'
sample='/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv'
categories=r'/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv'
train=r'/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv'
shops=r'/kaggle/input/competitive-data-science-predict-future-sales/shops.csv'
test=r'/kaggle/input/competitive-data-science-predict-future-sales/test.csv'

In [2]:
train=pd.read_csv(train)
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [3]:
shops=pd.read_csv(shops)
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [4]:
items=pd.read_csv(items)
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [5]:
categories=pd.read_csv(categories)
categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


# Merged DataFrame

In [6]:
df=pd.merge(train, shops, on='shop_id')
df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name
0,02.01.2013,0,59,22154,999.0,1.0,"Ярославль ТЦ ""Альтаир"""
1,10.01.2013,0,59,22151,399.0,1.0,"Ярославль ТЦ ""Альтаир"""
2,04.01.2013,0,59,5603,699.0,1.0,"Ярославль ТЦ ""Альтаир"""
3,19.01.2013,0,59,5587,199.0,2.0,"Ярославль ТЦ ""Альтаир"""
4,31.01.2013,0,59,5613,5571.0,1.0,"Ярославль ТЦ ""Альтаир"""


In [7]:
df.isnull().sum()

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
shop_name         0
dtype: int64

# Create New Date Features

In [8]:
df['Month'] = df['date'].str[3:5].astype(int)
months=list(df['Month'].unique())
print(months)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [9]:
df['Day'] = df['date'].str[:2].astype(int)
days=list(sorted(df['Day'].unique()))
print(days)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [10]:
df['Year'] = df['date'].str[6:10].astype(int)
years=list(sorted(df['Year'].unique()))
print(years)

[2013, 2014, 2015]


In [11]:
df.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,Month,Day,Year
0,02.01.2013,0,59,22154,999.0,1.0,"Ярославль ТЦ ""Альтаир""",1,2,2013
1,10.01.2013,0,59,22151,399.0,1.0,"Ярославль ТЦ ""Альтаир""",1,10,2013
2,04.01.2013,0,59,5603,699.0,1.0,"Ярославль ТЦ ""Альтаир""",1,4,2013
3,19.01.2013,0,59,5587,199.0,2.0,"Ярославль ТЦ ""Альтаир""",1,19,2013
4,31.01.2013,0,59,5613,5571.0,1.0,"Ярославль ТЦ ""Альтаир""",1,31,2013
5,10.01.2013,0,59,5623,699.0,1.0,"Ярославль ТЦ ""Альтаир""",1,10,2013
6,14.01.2013,0,59,5623,699.0,1.0,"Ярославль ТЦ ""Альтаир""",1,14,2013
7,10.01.2013,0,59,5629,2390.0,1.0,"Ярославль ТЦ ""Альтаир""",1,10,2013
8,04.01.2013,0,59,5643,2390.0,1.0,"Ярославль ТЦ ""Альтаир""",1,4,2013
9,17.01.2013,0,59,5643,2390.0,2.0,"Ярославль ТЦ ""Альтаир""",1,17,2013


# Test Data

In [12]:
testDF=pd.read_csv(test)
testDF.head(10)

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
5,5,5,5039
6,6,5,5041
7,7,5,5046
8,8,5,5319
9,9,5,5003


# Submit 

In [13]:
sample=pd.read_csv(sample)
sample.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
